This uses the XGBoost Ranking algorithm, which should be useful for a recommendation system but for testing purposes I've resorted to using the XGBoost linear regressor. The preprocessing in this code is not updated.

In [2]:
import pandas as pd
import numpy as np

import nltk
from nltk.corpus import stopwords
from brs_data_preprocessing import get_preprocessed_data as preproc, merged_book_ratings as merge

nltk.download('stopwords')


[nltk_data] Downloading package stopwords to
[nltk_data]     /home/legoeuro/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
usersDf, bookDf, ratingDf = preproc('input/Users.csv', 'input/Books.csv', 'input/Ratings.csv')

#configs
#Convert all strings to categorical (might not be good for title)
bookDf['Book-Title'] = bookDf['Book-Title'].astype('category')
bookDf['Book-Author'] = bookDf['Book-Author'].astype('category')
bookDf['Publisher'] = bookDf['Publisher'].astype('category')

ratingDf['ISBN'] = ratingDf['ISBN'].astype(str)
bookDf['ISBN'] = bookDf["ISBN"].astype(str)
ratingDf['User-ID'] = ratingDf['User-ID'].astype(int)
usersDf['User-ID'] = ratingDf['User-ID'].astype(int)

ratingDf = ratingDf.iloc[:20000]
bookDf = bookDf.iloc[:50000]

#(optional) split location into multiple categories (from string split by ',')
# for now I will just skip location altogether
usersDf.drop(columns=['Location'], inplace=True)

# usersDf['Location'] = usersDf['Location'].apply(lambda x: x.split(','))
# locationList = usersDf['Location'].to_list()
# flattenList = map(lambda x: x.strip(), [item for row in locationList for item in row])

In [4]:
bookDf.head()

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company


In [5]:
ratingDf.head()

,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6


In [6]:
usersDf.head()

,User-ID,Age
1,276726,18.0
3,276729,17.0
5,276733,61.0
9,276745,26.0
10,276746,14.0


Generate input for XGBoost

In [9]:
#reference: https://towardsdatascience.com/learning-to-rank-for-product-recommendations-a113221ad8a7
def generateInput(X_u, X_b, y):
    """
    X_u: User features
    X_m: Book features
    y: Ratings
    tgt_users: Target users
    """
    merged = pd.merge(X_u, y, on='User-ID', how='inner')
    merged = pd.merge(merged, X_b, on='ISBN', how='inner')

    merged.drop(columns=['ISBN'], inplace=True)

    # merged.fillna(0, inplace=True)
    merged = merged.dropna()
    return merged



In [10]:
from sklearn.model_selection import GroupShuffleSplit
#Split into train and test
df = generateInput(usersDf, bookDf, ratingDf)

gss = GroupShuffleSplit().split(df, groups=df['User-ID'])
train, test = next(gss)

train_data = df.iloc[train]
X_train = train_data.drop(columns=['Book-Rating', 'User-ID'])
y_train = train_data['Book-Rating']

test_data = df.iloc[test]
X_test = test_data.drop(columns=['Book-Rating'])
y_test = test_data['Book-Rating']

groups = train_data.groupby('User-ID').size().to_frame('size')['size'].to_numpy()
groups


array([  1, 221,   6, ...,   6,  50,  30])

In [11]:
y_train

13          5.0
14          5.0
15          5.0
16          5.0
17          5.0
           ... 
14611116    5.0
14611117    5.0
14611118    5.0
14611119    5.0
14611120    5.0
Name: Book-Rating, Length: 3736468, dtype: float64

In [ ]:
import xgboost as xgb

#tuneable parameters
model = xgb.XGBRanker(objective='rank:ndcg', n_estimators=100, random_state=0,learning_rate=0.1,
                      enable_categorical=True)


model.fit(X_train, y_train,  group=groups, verbose=True)

XGBRanker(base_score=None, booster=None, callbacks=None, colsample_bylevel=None,
          colsample_bynode=None, colsample_bytree=None, device=None,
          early_stopping_rounds=None, enable_categorical=True, eval_metric=None,
          feature_types=None, gamma=None, grow_policy=None,
          importance_type=None, interaction_constraints=None, learning_rate=0.1,
          max_bin=None, max_cat_threshold=None, max_cat_to_onehot=None,
          max_delta_step=None, max_depth=None, max_leaves=None,
          min_child_weight=None, missing=nan, monotone_constraints=None,
          multi_strategy=None, n_estimators=100, n_jobs=None,
          num_parallel_tree=None, random_state=0, ...)

Apply cosine vectorizer to all books; Find user top 10 books that they have not read

In [ ]:
def predict (model, df):
    return model.predict(df.loc[:, ~df.columns.isin(['User-ID'])])
  
predictions = (X_test.groupby('User-ID')
               .apply(lambda x: predict(model, x)))
predictions

/tmp/ipykernel_7567/4008277664.py:5: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda x: predict(model, x)))


User-ID
16.0            [-1.7457176, -1.634243, -2.035031, -2.204781]
22.0        [-2.0572004, -2.1904683, -2.1834931, -2.020229...
39.0        [-3.2873313, -3.4544895, -3.496933, -2.0658057...
53.0        [-3.5275736, -3.5275736, -3.6604211, -2.865101...
67.0        [-2.428505, -2.2839484, -2.3005881, -2.3111029...
                                  ...                        
278774.0    [-2.1939507, -2.1386213, -2.1386213, -2.048174...
278813.0                                         [-2.1935604]
278819.0                                         [-1.9607589]
278820.0                                         [-2.3529625]
278854.0    [-2.2929847, -2.2731333, -2.138771, -2.0048356...
Length: 266, dtype: object

In [ ]:
#evaluate model
from sklearn.metrics import mean_squared_error
from math import sqrt
rms = sqrt(mean_squared_error(y_test, predictions))

ValueError: Found input variables with inconsistent numbers of samples: [152480, 266]